In [ ]:
import pandas as pd

In [ ]:
weather = pd.read_csv("local_weather.csv", index_col= "DATE")

In [ ]:
weather

In [ ]:
weather.apply(pd.isnull).sum()/weather.shape[0]

In [ ]:
core_weather = weather[["PRCP","SNOW","SNWD","TMAX","TMIN"]].copy()

In [ ]:
core_weather.columns = ["precip","snow","snow_depth","temp_max","temp_min"]

In [ ]:
core_weather
print("core_weather on line 13",core_weather)

In [ ]:
core_weather.apply(pd.isnull).sum()/weather.shape[0]

In [ ]:
core_weather["snow"].value_counts()

In [ ]:
del core_weather["snow"]

In [ ]:
core_weather["snow_depth"].value_counts()

In [ ]:
del core_weather["snow_depth"]

In [ ]:
core_weather["precip"] = core_weather["precip"].fillna(0)

In [ ]:
core_weather = core_weather.fillna(method="ffill")

In [ ]:
core_weather.apply(pd.isnull).sum()/core_weather.shape[0]

In [ ]:
core_weather.dtypes

In [ ]:
core_weather.index

In [ ]:
core_weather.index = pd.to_datetime(core_weather.index)

In [ ]:
core_weather.index

In [ ]:
core_weather.index.year

In [ ]:
core_weather.apply(lambda x:(x==9999).sum())

In [ ]:
core_weather[["temp_max", "temp_min"]].plot()

In [ ]:
core_weather.index.year.value_counts().sort_index()

In [ ]:
core_weather["precip"].plot()

In [ ]:
core_weather.groupby(core_weather.index.year).sum()

In [ ]:
core_weather["target"] = core_weather.shift(-1)["temp_max"]

In [ ]:
core_weather
print("core_weather on line 27",core_weather)

In [ ]:
core_weather = core_weather.iloc[:-1,:].copy()

In [ ]:
core_weather


In [ ]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [ ]:
predictors = ["precip", "temp_max", "temp_min"]

In [ ]:
train = core_weather.loc[:"2022-12-31"]

In [ ]:
test = core_weather.loc["2023-01-01":]

In [ ]:
reg.fit(train[predictors], train["target"])

In [ ]:
predictions = reg.predict(test[predictors])

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(test["target"], predictions)

In [ ]:
combined = pd.concat([test["target"], pd.Series(predictions, index=test.index)], axis=1)
combined.columns = ["actual", "predictions"]

In [ ]:
combined

In [ ]:
combined.plot()

In [ ]:
reg.coef_

In [ ]:
def create_predictions(predictors, core_weather, reg):
    train = core_weather.loc[:"2022-12-31"]
    print("Line 49 predictors*********", predictors)
    print("train",train)
    test = core_weather.loc["2023-01-01":]
    print("test",test)
    reg.fit(train[predictors], train["target"])
    predictions = reg.predict(test[predictors])
    print("predictions",predictions)
    error = mean_absolute_error(test["target"], predictions)
    combined = pd.concat([test["target"], pd.Series(predictions, index=test.index)], axis=1)
    print("combined",combined)
    combined.columns = ["actual", "predictions"]
    return error, combined

In [ ]:
import numpy as np

# Check for NaNs
nan_counts = core_weather[predictors].isna().sum()
print("NaN counts in each predictor:\n", nan_counts)

# Check for Infinite values
inf_counts = np.isinf(core_weather[predictors]).sum()
print("Infinite counts in each predictor:\n", inf_counts)

# Identify rows containing NaN or Infinite values
problematic_rows = core_weather[predictors][core_weather[predictors].isna().any(axis=1) | np.isinf(core_weather[predictors]).any(axis=1)]
print("Rows with NaN or Infinite values:\n", problematic_rows)


In [ ]:
# Drop rows with NaN or Infinite values
core_weather_cleaned = core_weather.dropna().replace([np.inf, -np.inf], np.nan).dropna()
# core_weather_cleaned =0
print("core_weather_cleaned", core_weather_cleaned)

# Check again to ensure no NaNs or Infinities remain
nan_counts_after = core_weather_cleaned[predictors].isna().sum()
inf_counts_after = np.isinf(core_weather_cleaned[predictors]).sum()

print("After cleaning, NaN counts:\n", nan_counts_after)
print("After cleaning, Infinite counts:\n", inf_counts_after)


In [ ]:
core_weather_cleaned["month_max"] = core_weather_cleaned["temp_max"].rolling(30).mean()

In [ ]:
core_weather_cleaned

In [ ]:
core_weather_cleaned["month_day_max"] = core_weather_cleaned["month_max"] / core_weather_cleaned["temp_max"]

In [ ]:
core_weather["max_min"] = core_weather["temp_max"] /core_weather["temp_min"]

In [ ]:
predictors = ["precip", "temp_max", "temp_min", "month_max", "month_day_max", "max_min"]

In [ ]:
core_weather_cleaned["month_max"] = core_weather_cleaned["temp_max"].rolling(30).mean()
core_weather_cleaned["month_day_max"] = core_weather_cleaned["month_max"] / core_weather_cleaned["temp_max"]
core_weather_cleaned["max_min"] = core_weather_cleaned["temp_max"] / core_weather_cleaned["temp_min"]

In [ ]:
core_weather_cleaned

In [ ]:
core_weather_cleaned = core_weather_cleaned.iloc[30:,:].copy()

In [ ]:
error, combined = create_predictions(predictors, core_weather_cleaned, reg)
print(f"Mean Absolute Error: {error}")

In [ ]:
error

In [ ]:
combined.plot()

In [ ]:
core_weather["monthly_avg"] = core_weather["temp_max"].groupby(core_weather.index.month, group_keys=False).apply(lambda x: x.expanding(1).mean())


In [ ]:
core_weather

In [ ]:
core_weather["day_of_year_avg"] =  core_weather["temp_max"].groupby(core_weather.index.day_of_year, group_keys= False).apply(lambda x: x.expanding(1).mean())

In [ ]:
predictors = ["precip", "temp_max", "temp_min","month_day_max","max_min", "day_of_year_avg", "monthly_avg"]

In [ ]:
error, combined = create_predictions(predictors, core_weather, reg)

In [ ]:
error

In [ ]:
reg.coef_

In [ ]:
core_weather.corr()["target"]

In [ ]:
combined["diff"] = (combined["actual"] - combined["predictions"]).abs()

In [ ]:
combined.sort_values("diff", ascending = False).head()